In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

import nltk
import string
import re

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hafez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hafez\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
file_list = glob.glob(os.path.join(os.getcwd(), "imdb/train/pos", "*.txt"))
corpus_positive = [open(f, encoding="utf8").read() for f in file_list]

file_list = glob.glob(os.path.join(os.getcwd(), "imdb/train/neg","*.txt"))
corpus_negative = [open(f, encoding="utf8").read() for f in file_list]


In [13]:
df = pd.DataFrame(corpus_positive, columns=['text'])
df['sentiment'] = 'pos'

In [14]:
df2 = pd.DataFrame(corpus_negative, columns=['text'])
df2['sentiment'] = 'neg'

df = df.append(df2)
df

,text,sentiment
0,Bromwell High is a cartoon comedy. It ran at t...,pos
1,Homelessness (or Houselessness as George Carli...,pos
2,Brilliant over-acting by Lesley Ann Warren. Be...,pos
3,This is easily the most underrated film inn th...,pos
4,This is not the typical Mel Brooks film. It wa...,pos
...,...,...
12495,"Towards the end of the movie, I felt it was to...",neg
12496,This is the kind of movie that my enemies cont...,neg
12497,I saw 'Descent' last night at the Stockholm Fi...,neg
12498,Some films that you pick up for a pound turn o...,neg


In [28]:
sw = set(nltk.corpus.stopwords.words('english'))

def clean(text, stopwords):
    # remove tags like <tab>    
    text = re.sub(r'<[^<>]*>', ' ', text)  
    # split text on whitespace
    text_list = text.split()
    text_words = []
    
    punctuation = set(string.punctuation)
    
    # keep #tags and @mentions 
    ## punctuation.remove("#")
    ## punctuation.remove("@")
    
    for word in text_list:
        # remove punctuation marks at the beginning  
        # of each word
        while len(word) > 0 and word[0] in punctuation:
            word = word[1:]
        
        # remove punctuation marks at the end of each word
        while len(word) > 0 and word[-1] in punctuation:
            word = word[:-1]
        
        # a rule to eliminate most urls 
        if len(word) > 0 and "/" not in word:
            # eliminate stopwords 
            if word.lower() not in stopwords:
                # append the word to the text_words list
                text_words.append(word.lower())
        cleaner_text = " ".join(text_words)
    return cleaner_text

df['length'] = df.text.str.len()
df = df[df.length >= 100]
df.drop(['length'], axis=1, inplace=True)

print(df.iloc[0])
df['text'] = df['text'].apply(clean, stopwords=sw)
print(df.iloc[0])

text         Bromwell High is a cartoon comedy. It ran at t...
sentiment                                                  pos
Name: 0, dtype: object
text         bromwell high cartoon comedy ran time programs...
sentiment                                                  pos
Name: 0, dtype: object
